In [3]:
from robomaster import robot
import time
import csv
import os

# เก็บข้อมูล log
log_data_pos = []

# พารามิเตอร์ของ PD controller
Kp = 5.0                  # ค่า Proportional gain
Kd = 2.0                  # ค่า Derivative gain
setpoint = 2.0            # ระยะเป้าหมาย (เมตร)
current_x = 0.0           # ตำแหน่ง x ปัจจุบัน
prev_error = 0.0
prev_time = None

# เขียน CSV
def write_csv_once(file_path, fieldnames, entry):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if not file_exists or os.path.getsize(file_path) == 0:
            writer.writeheader()
        writer.writerow(entry)

# เก็บตำแหน่ง
def pos_info_handler(position_info):
    global current_x
    timestamp = time.time()
    current_x = position_info[0]  # อัปเดตตำแหน่ง x ปัจจุบัน

    entry = {"timestamp": timestamp, "x": position_info[0], "y": position_info[1], "z": position_info[2]}
    log_data_pos.append(entry)
    print(f"Position X: {position_info[0]:.2f}")
    write_csv_once("log_poshigh1.csv", ["timestamp", "x", "y", "z"], entry)

if __name__ == '__main__':
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    ep_chassis = ep_robot.chassis
    ep_chassis.sub_position(freq=10, callback=pos_info_handler)

    print("Starting PD-Control movement...")

    try:
        while True:
            current_time = time.time()
            error = setpoint - current_x

            # เวลาระหว่างรอบ (dt)
            if prev_time is None:
                dt = 0.1
            else:
                dt = current_time - prev_time

            derivative = (error - prev_error) / dt if dt > 0 else 0.0

            # PD Control
            speed = (Kp * error) + (Kd * derivative)

            # จำกัดช่วงความเร็ว
            speed = max(min(speed, 1.0), -1.0)

            if abs(error) <= 0.01:
                ep_chassis.drive_speed(x=0, y=0, z=0)
                print("Target distance reached.")
                break

            # สั่งการเคลื่อนที่
            ep_chassis.drive_speed(x=speed, y=0, z=0)

            # Debug info
            print(f"Error: {error:.2f}, Derivative: {derivative:.2f}, Speed: {speed:.2f}")

            # เตรียมรอบถัดไป
            prev_error = error
            prev_time = current_time

            time.sleep(0.1)

    finally:
        ep_chassis.unsub_position()
        ep_robot.close()
        print("Robot closed.")


Starting PD-Control movement...
Error: 2.00, Derivative: 20.00, Speed: 1.00
Position X: 0.00
Error: 2.00, Derivative: 0.00, Speed: 1.00
Position X: 0.00
Error: 2.00, Derivative: -0.04, Speed: 1.00
Position X: 0.04
Error: 1.96, Derivative: -0.37, Speed: 1.00
Position X: 0.13
Error: 1.87, Derivative: -0.78, Speed: 1.00
Position X: 0.23
Error: 1.77, Derivative: -0.91, Speed: 1.00
Position X: 0.32
Error: 1.68, Derivative: -0.85, Speed: 1.00
Position X: 0.40
Error: 1.60, Derivative: -0.67, Speed: 1.00
Position X: 0.47
Error: 1.53, Derivative: -0.63, Speed: 1.00
Position X: 0.55
Error: 1.45, Derivative: -0.73, Speed: 1.00
Position X: 0.63
Position X: 0.72Error: 1.37, Derivative: -0.81, Speed: 1.00

Position X: 0.80
Error: 1.20, Derivative: -1.51, Speed: 1.00
Position X: 0.87
Error: 1.13, Derivative: -0.69, Speed: 1.00
Position X: 1.00
Error: 1.00, Derivative: -1.13, Speed: 1.00
Position X: 1.08
Error: 0.92, Derivative: -0.76, Speed: 1.00
Position X: 1.16
Error: 0.84, Derivative: -0.72, Speed

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# อ่านไฟล์ CSV ทั้งสอง
df_high = pd.read_csv("log_poshigh.csv")
df_low = pd.read_csv("log_poslow.csv")

# ดึงข้อมูล timestamp และตำแหน่ง x
time_high = df_high['timestamp']
x_high = df_high['x']

time_low = df_low['timestamp']
x_low = df_low['x']

# สร้างกราฟ subplot แบบแนวตั้ง (2 แถว 1 คอลัมน์)
plt.figure(figsize=(10, 12))

# กราฟ High (แกน X = Time, แกน Y = Position)
plt.subplot(2, 1, 1)
plt.plot(time_high, x_high, color='red')
plt.xlabel("Time (s)")
plt.ylabel("Position X (m)")
plt.title("Position X over Time - High")
plt.grid(True)

# กราฟ Low (แกน X = Time, แกน Y = Position)
plt.subplot(2, 1, 2)
plt.plot(time_low, x_low, color='blue')
plt.xlabel("Time (s)")
plt.ylabel("Position X (m)")
plt.title("Position X over Time - Low")
plt.grid(True)

# จัด layout ให้ดูดี
plt.tight_layout()

# แสดงผล
plt.show()
